# データ読み込み

In [3]:
# アクティビティ内のevent取得
import re
from SPARQLWrapper import SPARQLWrapper
import pandas as pd
import math
from collections import Counter
from lib.episodes_loader import load_episodes_from_json
from lib.convert_answers import jsonify_answers

# def read_episods(file_path):
#     # csvからアクティビティ取得
#     episodes = []
#     with open("./Episods.csv", "r") as f:
#         lines = f.readlines()
#         for line in lines:
#             split_line = line.split(",")
#             scene = split_line[0]
#             day = split_line[1]
#             activities = list(filter(lambda x: x != '' and x != '\n', split_line[2:]))
#             activities = [activity.lower() for activity in activities]
#             episodes.append({"scene": scene, "day": day, "activities": activities})
    
#     return episodes

sparql = SPARQLWrapper(endpoint='https://morita.it.aoyama.ac.jp/homekg/sparql',
                        returnFormat='json')
# 削除するurlの定義
url_instance = "http://kgrc4si.home.kg/virtualhome2kg/instance/"
url_action = "http://kgrc4si.home.kg/virtualhome2kg/ontology/action/"
# event番号取得のパターン
event_num_pattern = re.compile(r'event([0-9]*)')

# jsonファイルからエピソードデータを読み込む
episodes = load_episodes_from_json("./data/Episodes.json")
"""
jsonファイルの場合，読み込まれるデータは以下の通り

id: エピソードのid 本番ではSenarioに該当
title: Day1など 本番では Morning Ritualsなど
scene: sceneのidのみが入る
activities: アクティビティの配列

"""

df = pd.DataFrame()
section_count = 0
columns = ["scene", "event_nums", "place", "action"]

for i, episode in enumerate(episodes):

    print(f"{episode['id']}")

    # それぞれのeventでsparqlを発行
    for activity in episode["activities"]:
        activity_name = (activity + "_scene" + str(episode["scene"])).strip()
        sparql.setQuery(f"""
            PREFIX ex: <http://kgrc4si.home.kg/virtualhome2kg/instance/>
            PREFIX : <http://kgrc4si.home.kg/virtualhome2kg/ontology/>
            select DISTINCT * where {{
                ex:{activity_name} :hasEvent ?event .
                ?event (:place | :from) ?nowplace.
                ?event :action ?action.
            }}
            """)
        results = sparql.query().convert()['results']['bindings']
        section_count += 1
        
        # 結果をまとめる
        results_summarized = []
        for result in results:
            event = result['event']
            place = result['nowplace']
            action = result['action']
            # イベント番号の取得
            event_num = event_num_pattern.match(event["value"].replace(url_instance, "")).groups()[0]
            results_summarized.append((
                int(event_num),
                place["value"].replace(url_instance, ""),
                action["value"].replace(url_action, "")
            ))
        # eventの番号でソート
        results_sorted = sorted(results_summarized, key=lambda x: x[0])

        scenes = []
        days = []
        event_nums = []
        places = []
        actions = []
        senarios = []

        for result in results_sorted:
            scenes.append("scene"+ str(episode["scene"]))
            days.append(episode["title"])
            event_nums.append(result[0])
            places.append(result[1])
            actions.append(result[2])
            senarios.append(episode["id"])
        
        df_results_outerloop = pd.DataFrame(columns=columns)
        df_results_outerloop["day"] = days
        df_results_outerloop["scene"] = scenes
        df_results_outerloop["event_nums"] = event_nums
        df_results_outerloop["place"] = places
        df_results_outerloop["action"] = actions
        df_results_outerloop["senario"] = senarios
        df = pd.concat([df, df_results_outerloop], ignore_index=True)

scene1_Day1
scene1_Day10
scene1_Day2
scene1_Day3
scene1_Day4
scene1_Day5
scene1_Day6
scene1_Day7
scene1_Day8
scene1_Day9
scene2_Day1
scene2_Day10
scene2_Day2
scene2_Day3
scene2_Day4
scene2_Day5
scene2_Day6
scene2_Day7
scene2_Day8
scene2_Day9
scene3_Day1
scene3_Day10
scene3_Day2
scene3_Day3
scene3_Day4
scene3_Day5
scene3_Day6
scene3_Day7
scene3_Day8
scene3_Day9
scene4_Day1
scene4_Day10
scene4_Day2
scene4_Day3
scene4_Day4
scene4_Day5
scene4_Day6
scene4_Day7
scene4_Day8
scene4_Day9
scene5_Day1
scene5_Day10
scene5_Day2
scene5_Day3
scene5_Day4
scene5_Day5
scene5_Day6
scene5_Day7
scene5_Day8
scene5_Day9
scene6_Day1
scene6_Day10
scene6_Day2
scene6_Day3
scene6_Day4
scene6_Day5
scene6_Day6
scene6_Day7
scene6_Day8
scene6_Day9
scene7_Day1
scene7_Day10
scene7_Day2
scene7_Day3
scene7_Day4
scene7_Day5
scene7_Day6
scene7_Day7
scene7_Day8
scene7_Day9


In [4]:
days = ["Day1", "Day2", "Day3", "Day4", "Day5", "Day6", "Day7", "Day8", "Day9", "Day10"]
scenes = ["scene1", "scene2", "scene3", "scene4", "scene5", "scene6", "scene7",]

In [5]:
df

,scene,event_nums,place,action,day,senario
0,scene1,0,bedroom75_scene1,walk,Day1,scene1_Day1
1,scene1,1,bedroom75_scene1,sit,Day1,scene1_Day1
2,scene1,3,bedroom75_scene1,walk,Day1,scene1_Day1
3,scene1,4,bedroom75_scene1,grab,Day1,scene1_Day1
4,scene1,5,bedroom75_scene1,put,Day1,scene1_Day1
...,...,...,...,...,...,...
7459,scene7,6,kitchen56_scene7,walk,Day9,scene7_Day9
7460,scene7,7,kitchen56_scene7,grab,Day9,scene7_Day9
7461,scene7,8,kitchen56_scene7,walk,Day9,scene7_Day9
7462,scene7,9,kitchen56_scene7,pour,Day9,scene7_Day9


In [16]:
def set_changes(df):

    df["change"] = False
    tmp = ""

    for i, item in df.iterrows():
        if i == 0:
            tmp = item["place"]
            df.loc[i, "change"] = True
            continue

        if tmp != item["place"]:
            df.loc[i, "change"] = True
        
        tmp = item["place"]
    
    return df

In [17]:
df_scene_day = dict()

for scene in scenes:
    tmp_dfs = dict()
    for day in days:
        tmp_df = df[(df["scene"]==scene) & (df["day"]==day)].reset_index()
        tmp_dfs[day] = set_changes(tmp_df)
    df_scene_day[scene] = tmp_dfs

df_scene_day["scene2"]["Day2"]

,index,scene,event_nums,place,action,day,senario,change
0,1366,scene2,0,kitchen51_scene2,walk,Day2,scene2_Day2,True
1,1367,scene2,1,kitchen51_scene2,grab,Day2,scene2_Day2,False
2,1368,scene2,2,kitchen51_scene2,walk,Day2,scene2_Day2,False
3,1369,scene2,3,kitchen51_scene2,put,Day2,scene2_Day2,False
4,1370,scene2,0,livingroom274_scene2,walk,Day2,scene2_Day2,True
5,1371,scene2,1,kitchen51_scene2,walk,Day2,scene2_Day2,True
6,1372,scene2,2,kitchen51_scene2,grab,Day2,scene2_Day2,False
7,1373,scene2,3,kitchen51_scene2,walk,Day2,scene2_Day2,False
8,1374,scene2,4,kitchen51_scene2,put,Day2,scene2_Day2,False
9,1375,scene2,5,kitchen51_scene2,switchOn,Day2,scene2_Day2,False


In [18]:
df_scene = dict()

for scene in scenes:
    tmp_df = df[df["scene"]==scene].reset_index()
    df_scene[scene] = set_changes(tmp_df)

df_scene["scene2"]

,index,scene,event_nums,place,action,day,senario,change
0,1210,scene2,0,livingroom274_scene2,walk,Day1,scene2_Day1,True
1,1211,scene2,1,livingroom274_scene2,grab,Day1,scene2_Day1,False
2,1212,scene2,2,livingroom274_scene2,walk,Day1,scene2_Day1,False
3,1213,scene2,3,kitchen51_scene2,walk,Day1,scene2_Day1,True
4,1214,scene2,4,bedroom197_scene2,walk,Day1,scene2_Day1,True
...,...,...,...,...,...,...,...,...
824,2034,scene2,3,livingroom274_scene2,walk,Day9,scene2_Day9,False
825,2035,scene2,0,kitchen51_scene2,walk,Day9,scene2_Day9,True
826,2036,scene2,1,kitchen51_scene2,grab,Day9,scene2_Day9,False
827,2037,scene2,2,kitchen51_scene2,walk,Day9,scene2_Day9,False


# 例外関数の処理

In [19]:
def do_exception1(tmp_place, now_place):
    if tmp_place == "toilet46_scene1" or now_place == "toilet46_scene1":
        return True
    else:
        return False

def do_exception4(tmp_place, now_place):
    if tmp_place == "toilet193_scene4" or now_place == "toilet193_scene4":
        return True
    else:
        return False

def do_exception5(tmp_place, now_place):
    if tmp_place == "toilet315_scene5" or now_place == "toilet315_scene5":
        return True
    else:
        return False

def do_exception6(tmp_place, now_place):
    if tmp_place == "toilet46_scene6" or now_place == "toilet46_scene6":
        return True
    else:
        return False


def check_exception(scene, places):
    if scene == "scene1":
        tmp_place, now_place = places[0], places[1]
        return do_exception1(tmp_place, now_place)
    elif scene == "scene4":
        tmp_place, now_place = places[0], places[1]
        return do_exception4(tmp_place, now_place)
    elif scene == "scene5":
        tmp_place, now_place = places[0], places[1]
        return do_exception5(tmp_place, now_place)
    elif scene == "scene6":
        tmp_place, now_place = places[0], places[1]
        return do_exception6(tmp_place, now_place)

# 瞬間移動の処理（一筆書き編）

In [20]:
# 瞬間移動の部屋を特定
def get_blank_room_list(tmp_place_index, now_place_index, rooms, scene, day):
    direction = ""
    if tmp_place_index < now_place_index:
        direction = "acen"
    elif now_place_index < tmp_place_index:
        direction = "desc"
    
    if direction == "acen":
        blank_rooms = [rooms[i] for i in range(tmp_place_index+1, now_place_index, 1)]
    elif direction == "desc":
        blank_rooms = [rooms[i] for i in range(tmp_place_index-1, now_place_index, -1)]

    blank_rooms_list = []
    for blank_room in blank_rooms:
        blank_rooms_list.append([scene, "-", blank_room, "-", day, True, "true", "-"])
    
    columns = ["scene", "event_nums", "place", "action", "day", "change", "blank", "transaction_id"]
    blank_rooms_df = pd.DataFrame(blank_rooms_list, columns=columns)
    return blank_rooms_df


# 瞬間移動の部屋を追加
def add_additional_blank_list(df, additional_blank_list):
    length = len(df)
    df_tmp = None
    for i, item in enumerate(additional_blank_list):
        transaction_id = item[0]
        length += item[1]
        additional_df = item[2]
        
        if i == 0:
            df_former = df[df["transaction_id"] <= transaction_id]    
        else:
            df_former = df[(df["transaction_id"] <= transaction_id) & (df["transaction_id"] > additional_blank_list[i-1][0])]

        df_tmp_pre = pd.concat([df_former, additional_df], ignore_index=True)
        df_tmp = pd.concat([df_tmp, df_tmp_pre], ignore_index=True)
        
    df_tmp_pre = df[df["transaction_id"] > transaction_id]
    df_tmp = pd.concat([df_tmp, df_tmp_pre], ignore_index=True)
    if len(df_tmp) == length:
        print("Completed!")
    else:
        print("Error occured!")
        print(length, len(df_tmp))
    
    return df_tmp


# 瞬間移動の有無を判定
def check_blank_oneStroke(df, rooms, noProblem_rooms):
    df["transaction_id"] = [i for i in range(len(df))]
    additional_blank_list = []
    isInit = 1
    tmp_place = ""

    for i, item in df.iterrows():
        if isInit == 1:
            isInit = 0
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
        
        now_place = item["place"]
        places = [tmp_place, now_place]

        # True なら continue
        if check_exception(item["scene"], places):
            tmp_place = item["place"]
            continue

        tmp_place_index = int(rooms.index(tmp_place))
        now_place_index = int(rooms.index(now_place))
        
        if tmp_place_index == now_place_index:
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
        
        flag = math.pow(2, tmp_place_index) + math.pow(2, now_place_index)
        if flag in noProblem_rooms:
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
        else:
            scene = item["scene"]
            day = item["day"]
            transaction_id = item["transaction_id"]-1

            blank_rooms_df = get_blank_room_list(tmp_place_index, now_place_index, rooms, scene, day)
            additional_blank_list.append([transaction_id, len(blank_rooms_df), blank_rooms_df])

            tmp_place = item["place"]
            df.loc[i, "blank"] = "just_after"
            continue
    
    return df, additional_blank_list

# 瞬間移動の処理（センター型編）

In [21]:
# 瞬間移動の有無を判定
def check_blank_center(df, rooms, center_room_id):
    center_room = rooms[center_room_id]
    other_rooms = [room for room in rooms if room != center_room]
    df["transaction_id"] = [i for i in range(len(df))]
    additional_blank_list = []
    isInit = 1

    for i, item in df.iterrows():
        if isInit == 1:
            isInit = 0
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
        
        now_place = item["place"]

        if tmp_place == now_place:
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
        
        if tmp_place in other_rooms and now_place in other_rooms:
            scene = item["scene"]
            day = item["day"]
            transaction_id = item["transaction_id"]-1            
    
            columns = ["scene", "event_nums", "place", "action", "day", "change", "blank", "transaction_id"]
            blank_rooms_df = pd.DataFrame([[scene, "-", center_room, "-", day, True, "true", "-"]], columns=columns)
            additional_blank_list.append([transaction_id, len(blank_rooms_df), blank_rooms_df])

            tmp_place = item["place"]
            df.loc[i, "blank"] = "just_after"
            continue
        else:
            tmp_place = item["place"]
            df.loc[i, "blank"] = "false"
            continue
    
    return df, additional_blank_list

In [22]:
def check_blank(df, rooms, shape_ofHouse, scene_noProblem_roomsIndex:None, center_room_id:None):
    if shape_ofHouse == "oneStroke":
        df, additional_blank_list = check_blank_oneStroke(df, rooms, scene_noProblem_roomsIndex)
    elif shape_ofHouse == "center":
        df, additional_blank_list = check_blank_center(df, rooms, center_room_id)
    
    if len(additional_blank_list) == 0:
        return df
    df = add_additional_blank_list(df, additional_blank_list)
    
    return df    

In [23]:
def get_enteredRoom_list(df):
    print(df)
    return Counter(df[df["change"]==True]["place"])

# 各 scene ごとの処理

## scene1

In [24]:
scene1_rooms = ["bathroom11_scene1", "bedroom75_scene1", "kitchen209_scene1", "livingroom342_scene1"]
scene1_noProblem_roomsIndex = [3, 6, 12]
df_scene1 = df_scene["scene1"]

df_scene1 = check_blank(df_scene1, scene1_rooms, "oneStroke", scene1_noProblem_roomsIndex, None)

get_enteredRoom_list(df_scene1)

Completed!
       index   scene event_nums                 place action   day  \
0        0.0  scene1          0      bedroom75_scene1   walk  Day1   
1        1.0  scene1          1      bedroom75_scene1    sit  Day1   
2        2.0  scene1          3      bedroom75_scene1   walk  Day1   
3        3.0  scene1          4      bedroom75_scene1   grab  Day1   
4        4.0  scene1          5      bedroom75_scene1    put  Day1   
...      ...     ...        ...                   ...    ...   ...   
1251  1205.0  scene1         22  livingroom342_scene1  scrub  Day9   
1252  1206.0  scene1         23  livingroom342_scene1  shake  Day9   
1253  1207.0  scene1         24  livingroom342_scene1    put  Day9   
1254  1208.0  scene1         25  livingroom342_scene1   walk  Day9   
1255  1209.0  scene1         26  livingroom342_scene1   wipe  Day9   

          senario  change transaction_id  blank  
0     scene1_Day1    True              0  false  
1     scene1_Day1   False              1  false 

Counter({'bedroom75_scene1': 99,
         'bathroom11_scene1': 40,
         'kitchen209_scene1': 107,
         'livingroom342_scene1': 48,
         'toilet46_scene1': 1})

## scene2

In [25]:
scene2_rooms = ["bedroom197_scene2", "kitchen51_scene2", "livingroom274_scene2"]
center_room_id = 1
df_scene2 = df_scene["scene2"]

df_scene2 = check_blank(df_scene2, scene2_rooms, "center", None, center_room_id)

get_enteredRoom_list(df_scene2)

Completed!
      index   scene event_nums                 place action   day  \
0    1210.0  scene2          0  livingroom274_scene2   walk  Day1   
1    1211.0  scene2          1  livingroom274_scene2   grab  Day1   
2    1212.0  scene2          2  livingroom274_scene2   walk  Day1   
3    1213.0  scene2          3      kitchen51_scene2   walk  Day1   
4    1214.0  scene2          4     bedroom197_scene2   walk  Day1   
..      ...     ...        ...                   ...    ...   ...   
832  2034.0  scene2          3  livingroom274_scene2   walk  Day9   
833  2035.0  scene2          0      kitchen51_scene2   walk  Day9   
834  2036.0  scene2          1      kitchen51_scene2   grab  Day9   
835  2037.0  scene2          2      kitchen51_scene2   walk  Day9   
836  2038.0  scene2          3      kitchen51_scene2    put  Day9   

         senario  change transaction_id  blank  
0    scene2_Day1    True              0  false  
1    scene2_Day1   False              1  false  
2    scene2_D

Counter({'livingroom274_scene2': 41,
         'kitchen51_scene2': 62,
         'bedroom197_scene2': 21})

## scene3

## scene4

In [26]:
scene4_rooms = ["bathroom177_scene4", "kitchen11_scene4", "bedroom216_scene4", "livingroom274_scene4"]
scene4_noProblem_roomsIndex = [3, 6, 12]
df_scene4 = df_scene["scene4"]

df_scene4 = check_blank(df_scene4, scene4_rooms, "oneStroke", scene4_noProblem_roomsIndex, None)

get_enteredRoom_list(df_scene4)

Completed!
      index   scene event_nums               place     action   day  \
0    3274.0  scene4          0  bathroom177_scene4       walk  Day1   
1    3275.0  scene4          1    kitchen11_scene4       walk  Day1   
2    3276.0  scene4          2   bedroom216_scene4       walk  Day1   
3    3277.0  scene4          3   bedroom216_scene4  switchOff  Day1   
4    3278.0  scene4          0    kitchen11_scene4       walk  Day1   
..      ...     ...        ...                 ...        ...   ...   
943  4196.0  scene4          4    kitchen11_scene4       walk  Day9   
944  4197.0  scene4          5    kitchen11_scene4       pour  Day9   
945  4198.0  scene4          6    kitchen11_scene4       pour  Day9   
946  4199.0  scene4          7    kitchen11_scene4        put  Day9   
947  4200.0  scene4          8    kitchen11_scene4        put  Day9   

         senario  change transaction_id  blank  
0    scene4_Day1    True              0  false  
1    scene4_Day1    True              

Counter({'bathroom177_scene4': 54,
         'kitchen11_scene4': 79,
         'bedroom216_scene4': 46,
         'livingroom274_scene4': 7,
         'toilet193_scene4': 18})

## scene5

In [27]:
scene5_rooms = ["bathroom295_scene5", "livingroom11_scene5", "kitchen112_scene5", "bedroom231_scene5"]
scene5_noProblem_roomsIndex = [3, 6, 12]
df_scene5 = df_scene["scene5"]

df_scene5 = check_blank(df_scene5, scene5_rooms, "oneStroke", scene4_noProblem_roomsIndex, None)

get_enteredRoom_list(df_scene5)

Completed!
       index   scene event_nums              place action   day      senario  \
0     4201.0  scene5          0  bedroom231_scene5   walk  Day1  scene5_Day1   
1     4202.0  scene5          1  bedroom231_scene5    sit  Day1  scene5_Day1   
2     4203.0  scene5          3  bedroom231_scene5   walk  Day1  scene5_Day1   
3     4204.0  scene5          4  bedroom231_scene5   grab  Day1  scene5_Day1   
4     4205.0  scene5          5  bedroom231_scene5    put  Day1  scene5_Day1   
...      ...     ...        ...                ...    ...   ...          ...   
1339  5490.0  scene5          9  kitchen112_scene5   walk  Day9  scene5_Day9   
1340  5491.0  scene5         10  kitchen112_scene5   grab  Day9  scene5_Day9   
1341  5492.0  scene5         11  kitchen112_scene5  drink  Day9  scene5_Day9   
1342  5493.0  scene5         12  kitchen112_scene5   walk  Day9  scene5_Day9   
1343  5494.0  scene5         13  kitchen112_scene5    put  Day9  scene5_Day9   

      change transaction_id 

Counter({'bedroom231_scene5': 49,
         'kitchen112_scene5': 116,
         'livingroom11_scene5': 108,
         'bathroom295_scene5': 46,
         'toilet315_scene5': 6})

## scene6

In [28]:
scene6_rooms = ["bedroom70_scene6", "kitchen161_scene6", "bathroom11_scene6", "livingroom260_scene6"]
center_room_id = 0
df_scene6 = df_scene["scene6"]

df_scene6 = check_blank(df_scene6, scene6_rooms, "center", None, center_room_id)

get_enteredRoom_list(df_scene6)

Completed!
       index   scene event_nums                 place     action   day  \
0     5495.0  scene6          0      bedroom70_scene6       walk  Day1   
1     5496.0  scene6          1      bedroom70_scene6        sit  Day1   
2     5497.0  scene6          3      bedroom70_scene6       walk  Day1   
3     5498.0  scene6          4      bedroom70_scene6       grab  Day1   
4     5499.0  scene6          5      bedroom70_scene6        put  Day1   
...      ...     ...        ...                   ...        ...   ...   
1000  6469.0  scene6         16  livingroom260_scene6        put  Day9   
1001  6470.0  scene6         17  livingroom260_scene6  switchOff  Day9   
1002     NaN  scene6          -      bedroom70_scene6          -  Day9   
1003  6471.0  scene6          0     bathroom11_scene6       walk  Day9   
1004  6472.0  scene6          1      bedroom70_scene6       walk  Day9   

          senario  change transaction_id       blank  
0     scene6_Day1    True              0     

Counter({'bedroom70_scene6': 116,
         'bathroom11_scene6': 36,
         'kitchen161_scene6': 40,
         'livingroom260_scene6': 45,
         'toilet46_scene6': 6})

## scene7

In [29]:
scene7_rooms = ["kitchen56_scene7", "bathroom11_scene7", "livingroom205_scene7"]
center_room_id = 0
df_scene7 = df_scene["scene7"]

df_scene7 = check_blank(df_scene7, scene7_rooms, "center", None, center_room_id)

get_enteredRoom_list(df_scene7)

     index   scene  event_nums             place     action   day  \
0     6473  scene7           0  kitchen56_scene7       walk  Day1   
1     6474  scene7           1  kitchen56_scene7       grab  Day1   
2     6475  scene7           2  kitchen56_scene7       walk  Day1   
3     6476  scene7           3  kitchen56_scene7       grab  Day1   
4     6477  scene7           4  kitchen56_scene7       walk  Day1   
..     ...     ...         ...               ...        ...   ...   
986   7459  scene7           6  kitchen56_scene7       walk  Day9   
987   7460  scene7           7  kitchen56_scene7       grab  Day9   
988   7461  scene7           8  kitchen56_scene7       walk  Day9   
989   7462  scene7           9  kitchen56_scene7       pour  Day9   
990   7463  scene7          10  kitchen56_scene7  switchOff  Day9   

         senario  change  transaction_id  blank  
0    scene7_Day1    True               0  false  
1    scene7_Day1   False               1  false  
2    scene7_Day1   Fa

Counter({'kitchen56_scene7': 65,
         'livingroom205_scene7': 63,
         'bathroom11_scene7': 1})

In [36]:
def get_enteredRoom_list(df):
    return Counter(df[df["change"]==True]["place"])

In [37]:
import re
# episodeごとに処理
days = ["Day1", "Day2", "Day3", "Day4", "Day5", "Day6", "Day7", "Day8", "Day9", "Day10"]
scenes = ["scene1", "scene2", "scene3", "scene4", "scene5", "scene6", "scene7",]

scene1_rooms = ["bathroom11_scene1", "bedroom75_scene1", "kitchen209_scene1", "livingroom342_scene1"]
scene2_rooms = ["bedroom197_scene2", "kitchen51_scene2", "livingroom274_scene2"]
scene4_rooms = ["bathroom177_scene4", "kitchen11_scene4", "bedroom216_scene4", "livingroom274_scene4"]
scene5_rooms = ["bathroom295_scene5", "livingroom11_scene5", "kitchen112_scene5", "bedroom231_scene5"]
scene6_rooms = ["bedroom70_scene6", "kitchen161_scene6", "bathroom11_scene6", "livingroom260_scene6"]
scene7_rooms = ["kitchen56_scene7", "bathroom11_scene7", "livingroom205_scene7"]

scene1_noProblem_roomsIndex = [3, 6, 12]
scene4_noProblem_roomsIndex = [3, 6, 12]
scene5_noProblem_roomsIndex = [3, 6, 12]

senarios = []
for scene in scenes:
    for day in days:
        senarios.append(scene+"_"+day)
        

for i, senario in enumerate(senarios):
    print(senario)
    answers = []
    tmp_df = df[df["senario"]==senario].reset_index()
    # tmp_df = df[df["scene"]==scene].reset_index()
    df_senario = set_changes(tmp_df)
    scene = re.sub("_.+", "", senario)
    if scene == "scene1":
        df_scene1 = check_blank(df_senario, scene1_rooms, "oneStroke", scene1_noProblem_roomsIndex, None)
        for room, count in get_enteredRoom_list(df_scene1).items():
            answers.append({"name": room, "number": count})
    if scene == "scene2":
        center_room_id = 1
        df_scene2 = check_blank(df_senario, scene2_rooms, "center", None, center_room_id)
        for room, count in get_enteredRoom_list(df_scene2).items():
            answers.append({"name": room, "number": count})
    #TODO: scene3のbedroomの重複修正
    # if scene == "scene3":
    #     center_room_id = 1
    #     df_scene2 = check_blank(df_senario, scene2_rooms, "center", None, center_room_id)
    # for room, count in get_enteredRoom_list(df_scene3).items():
    #     answers.append({"name": room, "number": count})
    if scene == "scene4":
        df_scene4 = df_scene["scene4"]
        df_scene4 = check_blank(df_senario, scene4_rooms, "oneStroke", scene4_noProblem_roomsIndex, None)
        for room, count in get_enteredRoom_list(df_scene4).items():
            answers.append({"name": room, "number": count})
    if scene == "scene5":
        df_scene5 = df_scene["scene5"]
        df_scene5 = check_blank(df_senario, scene5_rooms, "oneStroke", scene4_noProblem_roomsIndex, None)
        for room, count in get_enteredRoom_list(df_scene5).items():
            answers.append({"name": room, "number": count})
    if scene == "scene6":
        center_room_id = 0
        df_scene6 = df_scene["scene6"]
        df_scene6 = check_blank(df_senario, scene6_rooms, "center", None, center_room_id)
        for room, count in get_enteredRoom_list(df_scene6).items():
            answers.append({"name": room, "number": count})
    if scene == "scene7":
        center_room_id = 0
        df_scene7 = df_scene["scene7"]
        df_scene7 = check_blank(df_senario, scene7_rooms, "center", None, center_room_id)
        for room, count in get_enteredRoom_list(df_scene7).items():
            answers.append({"name": room, "number": count})  
            
    jsonify_answers(
        result_path="./result",
        answers=answers,
        senario=senario,
        question_id="q1",
    )


scene1_Day1
scene1_Day2
Completed!
scene1_Day3
Completed!
scene1_Day4
Completed!
scene1_Day5
Completed!
scene1_Day6
Completed!
scene1_Day7
Completed!
scene1_Day8
Completed!
scene1_Day9
Completed!
scene1_Day10
Completed!
scene2_Day1
Completed!
scene2_Day2
scene2_Day3
Completed!
scene2_Day4
Completed!
scene2_Day5
Completed!
scene2_Day6
Completed!
scene2_Day7
scene2_Day8
scene2_Day9
Completed!
scene2_Day10
scene3_Day1
scene3_Day2
scene3_Day3
scene3_Day4
scene3_Day5
scene3_Day6
scene3_Day7
scene3_Day8
scene3_Day9
scene3_Day10
scene4_Day1
Completed!
scene4_Day2
scene4_Day3
scene4_Day4
Completed!
scene4_Day5
Completed!
scene4_Day6
Completed!
scene4_Day7
Completed!
scene4_Day8
Completed!
scene4_Day9
Completed!
scene4_Day10
scene5_Day1
Completed!
scene5_Day2
Completed!
scene5_Day3
Completed!
scene5_Day4
Completed!
scene5_Day5
Completed!
scene5_Day6
Completed!
scene5_Day7
Completed!
scene5_Day8
Completed!
scene5_Day9
Completed!
scene5_Day10
Completed!
scene6_Day1
Completed!
scene6_Day2
Complete